In [2]:
@time using CUDA
@time using DelimitedFiles
@time using NearestNeighbors: KDTree, knn
@time using LinearAlgebra: norm

  0.000134 seconds (127 allocations: 11.359 KiB)
  0.000187 seconds (208 allocations: 21.656 KiB)
  0.000189 seconds (127 allocations: 11.375 KiB)
  0.000172 seconds (208 allocations: 21.656 KiB)


In [4]:
function NearNeighbor(X, t, t_knn)
    # Using kNN for Nearest Neighbors
    if t%t_knn | t == 0
        global kdtree = KDTree(X[:,1:3]')
    end 

    # Loop over all cells to compute displacements
    index, _ = knn(kdtree, X[1,1:3], 14, true)
    for i in 2:size(X)[1]
        # Scan neighbours
        global idxs, _ = knn(kdtree, X[i,1:3], 14, true)
        index = hcat(index, idxs)
    end
    return index
end

function force(X, r_max, s, K, NN)
    # Initialise displacement array
    global dX = zeros(Float64, size(X)[1], 3)

    # Loop over all cells to compute displacements
    for i in 1:size(X)[1]
        # Scan neighbours
        global idxs = NN[:,i]
        # Initialise variables
        global Xi = X[i,1:3]
        for j in idxs
            if i != j
                global r = Xi - X[j,1:3]
                global dist = norm(r)
                # Calculate attraction/repulsion force differential here
                if dist < r_max
                    global F = - K*(dist-r_max)*(dist-r_max)*(dist - s)
                    dX[i,:] +=  r/dist * F
                end
            end
        end
    end
    return dX
end

force (generic function with 1 method)

In [5]:
R_Agg = 16
t, t_knn = 0,10
# r_max, s = 2, 1
# K = 10
# dt = 0.5;

In [27]:
@time X = readdlm("../data/Init/Two_Sphere/$R_Agg.csv", ',', Float64, header=true)[1][:, 1:3]
@time X_Cu = CuArray{Float32}(X)

  0.036387 seconds (565.10 k allocations: 15.185 MiB)
  0.000089 seconds (8 allocations: 71.297 KiB)


6064×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -17.98  -2.89  -15.51
 -15.98  -2.89  -15.51
 -13.98  -2.89  -15.51
 -18.98  -1.15  -15.51
 -16.98  -1.15  -15.51
 -14.98  -1.15  -15.51
 -12.98  -1.15  -15.51
 -19.98   0.58  -15.51
 -17.98   0.58  -15.51
 -15.98   0.58  -15.51
   ⋮            
  16.98  -0.58   15.51
  18.98  -0.58   15.51
  13.98   1.15   15.51
  15.98   1.15   15.51
  17.98   1.15   15.51
  19.98   1.15   15.51
  14.98   2.89   15.51
  16.98   2.89   15.51
  18.98   2.89   15.51

In [14]:
@time NN = NearNeighbor(X, t, t_knn)
@time idx_Cu = CuArray{Int}(NN);

  0.331976 seconds (36.27 k allocations: 1.922 GiB, 16.60% gc time)
  0.000299 seconds (6 allocations: 192 bytes)


In [23]:
@time X_Cu

  0.000001 seconds


6064×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -17.98  -2.89  -15.51
 -15.98  -2.89  -15.51
 -13.98  -2.89  -15.51
 -18.98  -1.15  -15.51
 -16.98  -1.15  -15.51
 -14.98  -1.15  -15.51
 -12.98  -1.15  -15.51
 -19.98   0.58  -15.51
 -17.98   0.58  -15.51
 -15.98   0.58  -15.51
   ⋮            
  16.98  -0.58   15.51
  18.98  -0.58   15.51
  13.98   1.15   15.51
  15.98   1.15   15.51
  17.98   1.15   15.51
  19.98   1.15   15.51
  14.98   2.89   15.51
  16.98   2.89   15.51
  18.98   2.89   15.51